# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"UUID"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Se considera como **vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

### Cargar Librerías

In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

### Diccionarios para codificar variables

In [2]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

### Dataset de casos positivos

In [3]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})
# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas = df_cas.dropna()
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32', 'id_persona': 'int32'})

df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('float')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8') 

In [4]:
df_cas

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento
0,34978840,20200306,2,1,25,1,15.0
1,12149681,20200307,2,1,25,1,15.0
2,12211040,20200307,2,1,7,0,15.0
3,29640291,20200307,2,1,29,1,4.0
4,7800643,20200307,2,2,74,6,15.0
...,...,...,...,...,...,...,...
3464994,21286640,20220310,3,1,53,4,15.0
3464995,34343205,20220310,2,2,8,0,15.0
3464996,35199249,20220310,3,1,12,0,15.0
3464997,17161202,20220310,3,2,36,2,15.0


### Dataset de fallecidos

In [5]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'UUID']
df_fal = pd.read_csv(url_fal, sep = ';', usecols= col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32'}, 
                     low_memory=False)

In [6]:
df_fal = df_fal.dropna()  # Drop nan values
df_fal['fallecido'] = 1
df_fal = df_fal.astype({'UUID': 'int32', 'fallecido': 'int8'})

# Transformar a formato fecha para operar los días
df_fal['fecha'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha'].apply(lambda date : Week.fromdate(date) \
                                                                      .weektuple()) \
                                                                      .tolist()

# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento', 'UUID':'id_persona'}, inplace=True)

df_fal

,fecha_fallecimiento,id_persona,fallecido,fecha,year,epi_week
0,20210611,24833991,1,2021-06-11,2021,23
1,20210317,24761117,1,2021-03-17,2021,11
2,20210602,24767070,1,2021-06-02,2021,22
3,20210703,24751741,1,2021-07-03,2021,26
4,20210506,24671820,1,2021-05-06,2021,18
...,...,...,...,...,...,...
207660,20210514,18924935,1,2021-05-14,2021,19
207661,20210326,18847957,1,2021-03-26,2021,12
207662,20210603,18848024,1,2021-06-03,2021,22
207663,20210313,18848046,1,2021-03-13,2021,10


### Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [7]:
df = df_cas.merge(df_fal, on = 'id_persona', how = 'left')
df['fallecido'].fillna(0, inplace=True)
df['fallecido'] = df['fallecido'].astype('int8')
del df_fal, df_cas

# IMPORTANTE, se repiten pruebas en algunos
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fecha_fallecimiento,fallecido,fecha,year,epi_week
0,34978840,20200306,2,1,25,1,15.0,NaN,0,NaT,NaN,NaN
1,12149681,20200307,2,1,25,1,15.0,NaN,0,NaT,NaN,NaN
2,12211040,20200307,2,1,7,0,15.0,NaN,0,NaT,NaN,NaN
3,29640291,20200307,2,1,29,1,4.0,NaN,0,NaT,NaN,NaN
4,7800643,20200307,2,2,74,6,15.0,NaN,0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3465029,21286640,20220310,3,1,53,4,15.0,NaN,0,NaT,NaN,NaN
3465030,34343205,20220310,2,2,8,0,15.0,NaN,0,NaT,NaN,NaN
3465031,35199249,20220310,3,1,12,0,15.0,NaN,0,NaT,NaN,NaN
3465032,17161202,20220310,3,2,36,2,15.0,NaN,0,NaT,NaN,NaN


In [8]:
def days_between(d1, d2):
    return abs((d2 - d1).days)

# Agregar la diferencia en días desde el primer caso
fecha_min = df['fecha'].min()
fecha_max = df['fecha'].max() 

df['time_days'] = df['fecha'].apply(lambda row : days_between(row, fecha_min)+1)
df['time_days'] = df['time_days'].fillna(df.time_days.max()).astype('int')

df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento',
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week']]

date_life = Week.fromdate(fecha_max).weektuple()
df['year'] = df['year'].fillna(date_life[0]).astype('int')
df['epi_week'] = df['epi_week'].fillna(date_life[1]).astype('int')

/tmp/ipykernel_13471/568815843.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].fillna(date_life[0]).astype('int')
/tmp/ipykernel_13471/568815843.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['epi_week'] = df['epi_week'].fillna(date_life[1]).astype('int')


In [9]:
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,year,epi_week
0,34978840,20200306,2,1,25,1,15.0,0,727,NaN,2022,10
1,12149681,20200307,2,1,25,1,15.0,0,727,NaN,2022,10
2,12211040,20200307,2,1,7,0,15.0,0,727,NaN,2022,10
3,29640291,20200307,2,1,29,1,4.0,0,727,NaN,2022,10
4,7800643,20200307,2,2,74,6,15.0,0,727,NaN,2022,10
...,...,...,...,...,...,...,...,...,...,...,...,...
3465029,21286640,20220310,3,1,53,4,15.0,0,727,NaN,2022,10
3465030,34343205,20220310,2,2,8,0,15.0,0,727,NaN,2022,10
3465031,35199249,20220310,3,1,12,0,15.0,0,727,NaN,2022,10
3465032,17161202,20220310,3,2,36,2,15.0,0,727,NaN,2022,10


### Convertir en dask dataframe (ddf) para poder trabajar con el ddf de vacunados

In [10]:
df = dd.from_pandas(df, npartitions=3)

### Dataset de vacunados

In [11]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int32'})

# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
""" ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce")) """

ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,13474929,2,20211127,5
1,15456861,3,20220222,5
2,15428123,1,20211013,1
3,17787963,2,20211014,6
4,17873180,1,20220227,5


### Separar información de dosis recibidas para juntarlos con nuestro dataframe

In [12]:
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')
del df_dosis_3, ddf_vac

### Convertir nuestro ddf en un pandas dataframe

In [13]:
df = df.compute()

In [14]:
df.columns

Index(['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'time_days', 'fecha_fallecimiento', 'year',
       'epi_week', 'dosis_1', 'fecha_vacuna_1', 'id_vacuna_1', 'dosis_2',
       'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3', 'fecha_vacuna_3',
       'id_vacuna_3'],
      dtype='object')

### Rellenar espacios vacíos con 0

In [15]:
# Variable "vacunado" si recibe las 2 dosis
df['vacunado'] = np.where(df['dosis_2'] == 1, 1, 0).astype('int8')

df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

### Ordenar y Guardar el df

In [16]:
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento', 
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week',
         'vacunado', 'dosis_1', 'dosis_2', 'dosis_3', 
         'id_vacuna_1', 'id_vacuna_2', 'id_vacuna_3' ,'fecha_vacuna_1','fecha_vacuna_2', 'fecha_vacuna_3']]

### Eliminar duplicados

In [17]:
# Ordenar de acuerdo al id_de persona y fecha de resultado
df = df.sort_values( by = ['id_persona', 'fecha_resultado'])

In [18]:
# Vemos todos los duplicados, ya que debería haber un solo id para cada persona
df[df.id_persona.duplicated() == True]

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
69732,47,20220129,2,1,62,5,25.0,0,727,NaN,...,1,1,1,1,1,2,5,20210506.0,20210608.0,20211127.0
69794,89,20220129,2,2,74,6,25.0,0,727,NaN,...,1,1,1,0,1,2,0,20210426.0,20210609.0,NaN
37557,227,20210421,3,1,81,7,25.0,0,727,NaN,...,1,1,1,0,6,6,0,20211202.0,20211223.0,NaN
41765,324,20210525,3,2,54,4,25.0,0,727,NaN,...,1,1,1,1,2,1,5,20210824.0,20210927.0,20220106.0
68905,555,20220127,2,2,55,4,25.0,0,727,NaN,...,1,1,1,1,2,2,5,20210712.0,20210810.0,20211228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57061,37278364,20220116,3,2,1,0,20.0,0,727,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
31454,37281588,20210321,2,1,19,1,15.0,0,727,NaN,...,0,1,0,0,5,0,0,20220208.0,NaN,NaN
53381,37285950,20220112,2,1,26,1,20.0,1,54,20200506.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
53720,37286078,20220112,2,2,22,1,15.0,1,79,20200531.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


Podremos conservar el último duplicado, ya es es la fecha más reciente de un caso positivo. Estos duplicados representan varios contagios de la misma persona, sin embargo se toman como un nuevo contagio.

En este caso para el propósito del estudio nos interesa la mortalidad y la vacunación, por ello no se toma en cuenta el número de veces de contagio o fechas de la misma, si no todos los datos e id del paciente.

In [21]:
# Eliminamos los duplicados
df = df.drop_duplicates('id_persona', keep='last') \
       .sort_values( by = ['fecha_resultado']) \
       .reset_index(drop = True)

In [22]:
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
0,13708801,20200309,2,2,15,0,10.0,0,727,NaN,...,1,1,1,1,5,5,5,20210601.0,20210622.0,20211203.0
1,2020432,20200310,2,1,46,3,15.0,0,727,NaN,...,1,1,1,1,5,5,5,20210517.0,20210607.0,20211206.0
2,5493803,20200311,2,2,28,1,15.0,0,727,NaN,...,0,0,0,1,0,0,5,NaN,NaN,20220104.0
3,7979960,20200311,2,2,21,1,15.0,0,727,NaN,...,1,1,1,0,6,6,0,20210926.0,20211019.0,NaN
4,27127031,20200312,2,1,24,1,15.0,0,727,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247355,11628796,20220310,2,1,50,4,15.0,0,727,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3247356,21102513,20220310,2,1,33,2,15.0,0,727,NaN,...,1,1,1,0,6,6,0,20210904.0,20210925.0,NaN
3247357,4161824,20220310,2,2,39,2,14.0,0,727,NaN,...,1,1,1,0,6,6,0,20210909.0,20211004.0,NaN
3247358,1414645,20220310,2,1,55,4,15.0,0,727,NaN,...,1,1,1,1,5,5,5,20210630.0,20210721.0,20220107.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3247360 entries, 0 to 3247359
Data columns (total 22 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_persona           int32  
 1   fecha_resultado      int32  
 2   metododx             int8   
 3   sexo                 int8   
 4   edad                 int8   
 5   edad_cat             int8   
 6   departamento         float64
 7   fallecido            int8   
 8   time_days            int64  
 9   fecha_fallecimiento  float64
 10  year                 int64  
 11  epi_week             int64  
 12  vacunado             int8   
 13  dosis_1              int8   
 14  dosis_2              int8   
 15  dosis_3              int8   
 16  id_vacuna_1          int8   
 17  id_vacuna_2          int8   
 18  id_vacuna_3          int8   
 19  fecha_vacuna_1       float64
 20  fecha_vacuna_2       float64
 21  fecha_vacuna_3       float64
dtypes: float64(5), int32(2), int64(3), int8(12)
memory usage: 260.1 MB

In [24]:
df.to_csv('../Data/DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv',  index=False)